## 1 
Подготовить обучающую выборку, каждый элемент которой будет состоять из пар (X, D)m (m=1,…q) – обучающего вектора X = (x1,…,xn) (i=1,…,n) с вектором желаемых значений D = (d1,…,dk) (j=1,…,k) выходов персептрона.

In [1]:
from datareader import DataReader

train_dir = "data/train"
test_dir = "data/test"

input_height = 28
input_width = 28

input_channels = 1
num_classes = 6

train_generator = DataReader(train_dir, [input_height, input_width], True, input_channels, num_classes).get_generator()
test_generator = DataReader(test_dir, [input_height, input_width], False, input_channels, num_classes).get_generator()


print('Size of training set: {}'.format(train_generator.get_data_size()))
print('Size of testing set: {}'.format(test_generator.get_data_size()))

Size of training set: 24
Size of testing set: 6


## 2

Генератором случайных чисел всем синаптическим весам wij и нейронным смещениям w0,j (i=1,…,n; j=1,…,k) присваиваются некоторые малые случайные значения.


In [2]:
from pure_python.neuron import Neuron
from random import random
from pprint import pprint

inputs_count = input_height * input_width

range_min = - 0.0003
range_max = 0.0003


def generate_weights():
    weights = []
    for i in range(inputs_count+1):
        weights.append(range_min + (range_max - range_min) * random())
    return weights
    

pprint(generate_weights())

[0.00026331440135752264,
 4.121500476302758e-06,
 -0.00013911455555044026,
 0.00018314001500737543,
 0.00012035860852553621,
 -0.00013855638420431918,
 -0.00018960334894790908,
 -2.8100696875512027e-06,
 -1.0793714433560099e-05,
 -5.542712359221831e-05,
 0.0002081724662036214,
 6.493183555128534e-05,
 0.00024499871513962136,
 9.028542913215592e-05,
 -0.00025299170785817056,
 7.471308648350586e-05,
 -0.000143963477352156,
 -9.224723333004176e-05,
 -0.00029808815718225713,
 0.00021131306497128207,
 0.00013384663660882421,
 0.00020824376639338175,
 0.00011412161529947046,
 0.00025403993630512904,
 0.00011808683705200751,
 -0.0002602703970319163,
 -0.000273113347141768,
 0.00028369393611177377,
 -3.474521336590023e-05,
 -3.863369093615529e-05,
 -0.00019960659828034054,
 3.628144954602073e-05,
 -0.0001602232140336816,
 5.1310906800607525e-05,
 -0.00023356560210201986,
 0.00021559715174264573,
 9.806340058484119e-05,
 -0.00024270216532456336,
 0.00011922387041350186,
 0.00023340859625564457,

## 3
Из обучающей выборки (X, D)1,…,(X, D)q, взять следующий по счету вектор Xт = (x1,…,xn) и подать его на входы персептрона x1,…,xn. Сигналам нейронных входов смещения x0 присваиваются единичные значения: x0 = 1.

In [3]:
from pure_python._vector import Vector

x, d = train_generator.next()
vector = Vector(x, d)

neurons = [Neuron(inputs_count) for i in range(num_classes)]

for neuron in neurons:
    neuron.set_weights(generate_weights())

pprint(neurons)

## 4
Шаг 4. Для каждого j-го нейрона вычислить взвешенную сумму входных сигналов netj и выходной сигнал yj на основании функции активации f:
netj=i=0nwi,jxi
yj=fnetj


In [4]:
for neuron in neurons:
    neuron.delta_calc_y(vector.get_x())
    print(f"y: {neuron.get_y()}")
    print(f"net: {neuron.get_net()}")

y: 0.0
net: 0.0
y: 1.0
net: 0.0
y: 0.0
net: 0.0
y: 0.0
net: 0.0
y: 0.0
net: 0.0
y: 0.0
net: 0.0


## 5
Для каждого j-го нейрона вычислить его ошибку

In [5]:

for j, neuron in enumerate(neurons):
    loss = vector.get_d()[j] - neuron.get_y()
    print(f"{j}: loss = {loss}")

0: loss = 1.0
1: loss = -1.0
2: loss = 0.0
3: loss = 0.0
4: loss = 0.0
5: loss = 0.0


## 6
Произвести коррекцию синаптических весов j-го нейрона и нейронных смещений

In [7]:
eta = 0.5

for i, neuron in enumerate(neurons):
    neuron_weights = neuron.get_weights()
    delta_weights = [0] * (len(neuron_weights) + 1)

    for j, weight in enumerate(neuron_weights):
        delta_weights[j] += eta * loss * vector.get_x()[j]
    neuron.correct_weights(delta_weights)

## 7
Шаги 4-6 повторяются для всех нейронов персептронного слоя при подаче конкретного образа.

Шаги 3-6 выполняются последовательно для каждого входного образа, на котором обучается персептрон.

Шаг 7. После подачи последнего обучающего вектора, проверить критерий останова обучения, если он выполняется, то завершить обучение. В противном случае – возврат к шагу 3.

In [26]:
train_generator.reset_pointer()
train_generator.shuffle_data()

num_epochs = 150

neurons = [Neuron(inputs_count) for i in range(num_classes)]

for neuron in neurons:
    neuron.set_weights(generate_weights())

    
neu_loss = 0
    
for epoch in range(num_epochs):
    neu_loss = 0
    x, d = train_generator.next()
    vector = Vector(x, d)
    for neuron in neurons:
        neuron.delta_calc_y(vector.get_x())
    errors = [0] * (len(neurons)+1)
    error = 0.0

    for i, neuron in enumerate(neurons):
        errors[i] = vector.get_d()[i] - neuron.get_y()
        error += errors[i]

    for i, neuron in enumerate(neurons):
        neuron_weights = neuron.get_weights()
        delta_weights = [0] * (len(vector.get_x())+1)

        for j, weight in enumerate(neuron_weights):
            delta_weights[j] += eta * errors[i] * vector.get_x()[j]
        neuron.correct_weights(delta_weights)

            
            
    train_generator.reset_pointer()
    train_generator.shuffle_data()
    
    print(f"epoch: {epoch+1}, error: {error}")

epoch: 1, error: -2.0
epoch: 2, error: -2.0
epoch: 3, error: 0.0
epoch: 4, error: -1.0
epoch: 5, error: 0.0
epoch: 6, error: 0.0
epoch: 7, error: 1.0
epoch: 8, error: 0.0
epoch: 9, error: 0.0
epoch: 10, error: 0.0
epoch: 11, error: 0.0
epoch: 12, error: 0.0
epoch: 13, error: 0.0
epoch: 14, error: 0.0
epoch: 15, error: -1.0
epoch: 16, error: 0.0
epoch: 17, error: 0.0
epoch: 18, error: 0.0
epoch: 19, error: 0.0
epoch: 20, error: -1.0
epoch: 21, error: -1.0
epoch: 22, error: 1.0
epoch: 23, error: 0.0
epoch: 24, error: 0.0
epoch: 25, error: 0.0
epoch: 26, error: -1.0
epoch: 27, error: 0.0
epoch: 28, error: 1.0
epoch: 29, error: 0.0
epoch: 30, error: -1.0
epoch: 31, error: 0.0
epoch: 32, error: 0.0
epoch: 33, error: 1.0
epoch: 34, error: 0.0
epoch: 35, error: 0.0
epoch: 36, error: 0.0
epoch: 37, error: -1.0
epoch: 38, error: 0.0
epoch: 39, error: 0.0
epoch: 40, error: 0.0
epoch: 41, error: 0.0
epoch: 42, error: 0.0
epoch: 43, error: 0.0
epoch: 44, error: -1.0
epoch: 45, error: 0.0
epoch: 46

## 8
Test

In [27]:
test_generator.reset_pointer()
def get_max_neuron_idx(neurons):
    max_idx = -1
    answer = -1
    for j in range(len(neurons)):
        if neurons[j] > answer:
            answer = neurons[j]
            max_idx = j
    return max_idx

for i in range(test_generator.get_data_size()):
    x, d = test_generator.next()
    vector = Vector(x, d)
    y = [0] * len(neurons)
    for j, neuron in enumerate(neurons):
        neuron.delta_calc_y(vector.get_x())
        y[j] = neuron.get_y()

    print(f"{get_max_neuron_idx(d)} recognized as {get_max_neuron_idx(y)}")

0 recognized as 0
1 recognized as 1
2 recognized as 2
3 recognized as 3
4 recognized as 4
5 recognized as 5
